# 3D Solid Mechanics

In [2]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import ngsolve

In [3]:
box = Box((0,0,0), (3,0.6,1))
box.faces.name="outer"
cyl = sum( [Cylinder((0.5+i,0,0.5), Y, 0.25,0.8) for i in range(3)] )
cyl.faces.name="cyl"
geo = box-cyl

DrawGeo(geo);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

find edges between box and cylinder, and build chamfers (requires OCC 7.4 or newer):

In [4]:
cylboxedges = geo.faces["outer"].edges * geo.faces["cyl"].edges
cylboxedges.name = "cylbox"
geo = geo.MakeChamfer(cylboxedges, 0.03)

name faces for boundary conditions:

In [5]:
geo.faces.Min(X).name = "fix"
geo.faces.Max(X).name = "force"

DrawGeo(geo);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

In [6]:
from ngsolve import *
from ngsolve.webgui import Draw
import pyngcore as ngcore
ngcore.SetNumThreads(8)
with ngcore.TaskManager():
    mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.01)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## Linear elasticity

Displacement: $u : \Omega \rightarrow R^3$

Linear strain:
$$
\varepsilon(u) := \tfrac{1}{2} ( \nabla u + (\nabla u)^T )
$$

Stress by Hooke's law:
$$
\sigma = 2 \mu \varepsilon + \lambda \operatorname{tr} \varepsilon I
$$

Equilibrium of forces:
$$
\operatorname{div} \sigma = f
$$

Displacement boundary conditions:
$$
u = u_D \qquad \text{on} \, \Gamma_D
$$

Traction boundary conditions:
$$
\sigma n = g \qquad \text{on} \, \Gamma_N
$$


Variational formulation:
--- 
Find: $u \in H^1(\Omega)^3$ such that $u = u_D$ on $\Gamma_D$
$$
\int_\Omega \sigma(\varepsilon(u)) : \varepsilon(v) \, dx = \int_\Omega f v dx + \int_{\Gamma_N} g v ds
$$
holds for all $v = 0$ on $\Gamma_D$.

In [7]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)    

In [8]:
fes = VectorH1(mesh, order=3, dirichlet="fix")
u,v = fes.TnT()
gfu = GridFunction(fes)

with TaskManager():
    a = BilinearForm(InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx)
    pre = Preconditioner(a, "bddc")
    a.Assemble()

KeyboardInterrupt: 

In [ ]:
force = CF( (1e-3,0,0) )
f = LinearForm(force*v*ds("force")).Assemble()

In [ ]:
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre, printrates=True, tol=1e-8)
gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.0001798273150819818     
CG iteration 2, residual = 7.527505823604729e-05     
CG iteration 3, residual = 8.667360647588635e-05     
CG iteration 4, residual = 6.611763953432876e-05     
CG iteration 5, residual = 6.009530495082669e-05     
CG iteration 6, residual = 4.240151181641356e-05     
CG iteration 7, residual = 3.17465966410974e-05     
CG iteration 8, residual = 2.700717685978023e-05     
CG iteration 9, residual = 2.0093499070757786e-05     
CG iteration 10, residual = 1.6202784652563635e-05     
CG iteration 11, residual = 1.1164241035834833e-05     
CG iteration 12, residual = 8.707361209402886e-06     
CG iteration 13, residual = 6.263300218970041e-06     
CG iteration 14, residual = 4.827662227890135e-06     
CG iteration 15, residual = 3.504830182391952e-06     
CG iteration 16, residual = 2.6744872507300332e-06     
CG iteration 17, residual = 1.9959626253875047e-06     
CG iteration 18, residual = 1.4173939615005607e-06     
CG iteration 1

In [ ]:
with TaskManager():
    fesstress = MatrixValued(H1(mesh,order=3), symmetric=True)
    gfstress = GridFunction(fesstress)
    gfstress.Interpolate (Stress(Sym(Grad(gfu))))

In [ ]:
Draw (5e4*gfu, mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [ ]:
Draw (Norm(gfstress), mesh, deformation=1e4*gfu, draw_vol=False, order=3);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…